#  Algoritmo de PLN

# 1. Setup

Realiza o download e configura o ambiente para o desenvolvimento do algoritmo PLN


**Bibilioteca ScikitLearn**

Utilizada para pré-processamento, análise de dados e modelagem preditiva



In [33]:
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

**Numpy**
Fornece suporte para arrays multidimensionais e operações matemáticas rápidas

In [34]:
import numpy as np

**Pandas**

Manipulação e análise de dados

In [35]:
import pandas as pd

**Gdown**

Ferramenta para baixar arquivos do Google Drive para o ambiente local

In [36]:
import gdown

**Pickle**

Permite salvar e carregar objetos python

In [37]:
import pickle

**Biblioteca Spacy e Natural Language Toolkit (NLTK)**

Utilizadas para processamento de linguagem natural

In [38]:
import spacy

spacy.cli.download('pt_core_news_lg')
nlp = spacy.load('pt_core_news_lg')

✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [39]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import pos_tag, ne_chunk

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

**Regex**

Ferramenta para manipulação e análise de strings

In [40]:
import re

**Transformers**

Biblioteca da Hugging Face para modelos de aprendizado profundo em NLP.

In [41]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

**Torch**

Biblioteca de aprendizado profundo desenvolvida pelo Facebook

In [42]:
import torch
from torch.utils.data import Dataset

**Gensim**

Utilizada para modelagem de tópicos e análise semântica

In [43]:
from gensim.models import KeyedVectors

Baixando o arquivo do modelo para o word2vec

In [44]:
gdown.download('https://drive.google.com/uc?id=1htT3BRNWNPBkX965rgGNuvz7jShrebfH', 'slip_s300.txt', quiet=False)

caminho_modelo = 'slip_s300.txt'

Downloading...
From (original): https://drive.google.com/uc?id=1htT3BRNWNPBkX965rgGNuvz7jShrebfH
From (redirected): https://drive.google.com/uc?id=1htT3BRNWNPBkX965rgGNuvz7jShrebfH&confirm=t&uuid=ece70d03-ff18-4335-988f-66e7095b4bb9
To: /content/slip_s300.txt
100%|██████████| 2.66G/2.66G [00:50<00:00, 52.5MB/s]


Configuração da frase que será utilizada como input em todo o notebook.

In [45]:
texto = 'Todas regulamentações de comunhão de recuros destinados à aplicação de investimentos da Anbima de 23/02/2003'

# 2. Download do dataset

Download do dataset com dados sintéticos

In [46]:
gdown.download('https://drive.google.com/uc?id=1W7iU_vqGMjWooWF4wpDy_vkfR2DjqmYq', 'dados.csv', quiet=False)
dados = pd.read_csv('dados.csv')

Downloading...
From: https://drive.google.com/uc?id=1W7iU_vqGMjWooWF4wpDy_vkfR2DjqmYq
To: /content/dados.csv
100%|██████████| 22.0k/22.0k [00:00<00:00, 32.8MB/s]


In [47]:
df_dados = pd.DataFrame(dados)
df_dados.head()

,frase,classificacao
0,Quero todas as regulamentações sobre fundos de...,Fundos de investimento
1,Quais são as normas para criação de fundos de ...,Fundos de investimento
2,Preciso das regulamentações sobre fundos imobi...,Fundos de investimento
3,Informações sobre fundos de índice.,Fundos de investimento
4,Quero as regras para fundos de ações.,Fundos de investimento


# 3. Modelo Random Forest


## 3.1 Dataset

Criando uma cópia do dataset anterior para utilizar no pré-processamento do modelo Random Forest

In [83]:
df_modelo1 = df_dados.copy()
df_modelo1.head()

,frase,classificacao
0,Quero todas as regulamentações sobre fundos de...,Fundos de investimento
1,Quais são as normas para criação de fundos de ...,Fundos de investimento
2,Preciso das regulamentações sobre fundos imobi...,Fundos de investimento
3,Informações sobre fundos de índice.,Fundos de investimento
4,Quero as regras para fundos de ações.,Fundos de investimento


## 3.2 Pré-Processamento

O pré-processamento é a etapa em que envolve transformar os dados em um formato adequado para o modelo preditivo processar.

### 3.2.1 Filtro
Esta etapa remove datas e pontuações do texto. Primeiro, é utilizada uma expressão regular para encontrar e remover datas no formato dd/mm/aaaa. Em seguida, são removidas as pontuações, resultando em um texto somente com letras.

In [49]:
import string
def remover_pontuacao_e_datas(texto):
    padrao_data = r'\b\d{1,2}/\d{1,2}(?:/\d{2,4})?\b'

    texto_sem_datas = re.sub(padrao_data, '', texto)

    texto_sem_pontuacao = texto_sem_datas.translate(str.maketrans('', '', string.punctuation))

    somente_texto = ' '.join(texto_sem_pontuacao.split())

    return somente_texto

#### Teste da função de filtro

In [50]:
teste_somente_texto = remover_pontuacao_e_datas('O rato roeu a roupa do rei de Roma em 14/04/2004.')
print(teste_somente_texto)

if teste_somente_texto == 'O rato roeu a roupa do rei de Roma em':
    print("Teste passou!")
else:
    print("Teste falhou.")

O rato roeu a roupa do rei de Roma em
Teste passou!


### 3.2.2 Minúsculo

Esta etapa transforma todo o texto em letras minúsculas, garantindo consistência no tratamento das palavras.

In [51]:
def texto_para_minusculo(texto):
    return texto.lower()

####Teste da etapa

In [52]:
resultado_minusculo = texto_para_minusculo('O rato roeu a roupa do rei de Roma')

if resultado_minusculo == "o rato roeu a roupa do rei de roma":
    print("Teste passou!")
else:
    print("Teste falhou.")

Teste passou!


### 3.2.3 Tokenização

 Etapa para realizar a tokenização do texto, dividindo-o em uma lista de palavras (tokens), que facilita o processamento de forma individual

In [53]:
def tokenizar_texto(texto):
    return word_tokenize(texto)

####Teste da etapa

In [54]:
teste_tokens = tokenizar_texto('O rato roeu a roupa do rei de Roma')
print(teste_tokens)

if teste_tokens == ['O', 'rato', 'roeu', 'a', 'roupa', 'do', 'rei', 'de', 'Roma']:
    print("Teste passou!")
else:
    print("Teste falhou.")

['O', 'rato', 'roeu', 'a', 'roupa', 'do', 'rei', 'de', 'Roma']
Teste passou!


### 3.2.4 StopWords

Etapa que remove stopwords do texto, eliminando palavras comuns e irrelevantes como "e", "o", "de".

In [55]:
def remover_stopwords(texto):
  stop_words = set(stopwords.words('portuguese'))
  return [word for word in texto if word.lower() not in stop_words]

#### Teste da etapa

In [56]:
teste_stopwords = remover_stopwords(teste_tokens)
print('Sem stopwords:', teste_stopwords)

if teste_stopwords == ['rato', 'roeu', 'roupa', 'rei', 'Roma']:
    print("Teste passou!")
else:
    print("Teste falhou.")

Sem stopwords: ['rato', 'roeu', 'roupa', 'rei', 'Roma']
Teste passou!


### 3.2.5 Extrair Entidades Regulamentadoras

Identificação de tokens que correspondem aos nomes dos reguladores financeiros.

A identificação dos reguladores é realizada através de uma expressão regularque é gerada para encontrar termos específicos que corrrespondem ao dicionário criado, retornando os reguladores encontrados no texto.

In [57]:
def extrair_reguladores(tokens):
    dicionario = {"anbima", "nuclea", "bacen", "cvm", "b3", "selic", "banco do brasil", "anpd", "bsm", "febraban", "apimec", "abbc", "coaf"}
    padrao = r'\b(?:' + '|'.join(re.escape(palavra) for palavra in dicionario) + r')\b'

    if isinstance(tokens, str):
        doc = nlp(tokens)
        tokens = [token.text for token in doc]

    reguladores_encontrados = [token for token in tokens if re.search(padrao, token.lower())]

    return reguladores_encontrados

####Teste da etapa

Teste com tokens

In [58]:
teste_reguladores_tokens = extrair_reguladores(['todas', 'as', 'regulamentações', 'do', 'bacen'])
print('Reguladores:', teste_reguladores_tokens)

if teste_reguladores_tokens == ['bacen']:
    print("Teste passou!")
else:
    print("Teste falhou.")

Reguladores: ['bacen']
Teste passou!


Teste com frase

In [59]:
teste_reguladores_string = extrair_reguladores('todas as regulamentações do bacen')
print('Reguladores:', teste_reguladores_string)

if teste_reguladores_string == ['bacen']:
    print("Teste passou!")
else:
    print("Teste falhou.")

Reguladores: ['bacen']
Teste passou!


### 3.2.6 Extrair Datas

Etapa para encontras as datas citadas no texto

 Utiliza uma expressão regular para encontrar todas as datas no formato dd/mm/aaaa ou dd/mm em um texto.

In [60]:
def encontrar_datas(texto):
    padrao_data = r'\b\d{1,2}/\d{1,2}(?:/\d{2,4})?\b'
    datas = re.findall(padrao_data, texto)
    return datas

#### Teste da etapa

In [61]:
teste_datas = encontrar_datas('O rato roeu a roupa do rei de Roma em 14/04/2004 e depois em 12/08 do mesmo ano.')
print('Datas:', teste_datas)

if teste_datas == ['14/04/2004', '12/08']:
    print("Teste passou!")
else:
    print("Teste falhou.")

Datas: ['14/04/2004', '12/08']
Teste passou!


### 3.2.7 Vetorização
Etapa para encontrar o vetor das frases, utilizando um modelo pré-treinado Word2Vec, que mapeia palavras para seus vetores correspondentes.


Carregando o modelo:

In [63]:
model_w2v = KeyedVectors.load_word2vec_format('slip_s300.txt', binary=False)

Salva o modelo Word2Vec em um arquivo, permitindo que o modelo seja carregado posteriormente

In [64]:
with open('model_w2v.pkl', 'wb') as f:
    pickle.dump(model_w2v, f)

A lógica encontrada na função gera a média dos vetores de palavras (tokens) de um texto usando o modelo. Se nenhuma palavra for encontrada no modelo, é retornado um vetor nulo.

In [65]:
def vetorizar(tokens, model):
    vectors = [model[token] for token in tokens if token in model]

    if len(vectors) > 0:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

####Teste da etapa

In [66]:
teste_vetorizar = vetorizar(['rato', 'roeu', 'roupa', 'rei', 'roma'], model_w2v)

vetor_esperado = np.array([0.011315, -0.0205364, -0.17001821, 0.0281516, -0.0009176, 0.0048138,
 0.18003121, -0.080958, 0.10080741, 0.0004238, -0.1151844, -0.19962598,
 0.0086752, 0.1176168, -0.1799024, -0.0301898, -0.0458002, 0.13258919,
 -0.2186378, -0.0577094, -0.0155758, -0.1074546, 0.0311892, 0.0110948,
 -0.0331072, -0.0066584, -0.0006306, 0.0363484, -0.03908, -0.0272782,
 0.0033082, 0.12251041, -0.0291818, -0.0924454, 0.1087174, -0.0194612,
 -0.1306488, -0.0064442, -0.14920822, 0.0102678, 0.0451484, 0.12865861,
 0.0048968, 0.0209104, 0.1455828, 0.1445082, 0.0672162, -0.14650841,
 0.0122242, 0.0641984, -0.14387742, -0.12351298, -0.0743102, 0.010645,
 -0.0058324, 0.1283564, 0.17453358, -0.0113002, 0.07685401, -0.0293362,
 0.235621, 0.1729084, -0.0046264, 0.2931738, 0.0711944, 0.1012754,
 0.0191284, -0.0504944, -0.056211, 0.11169519, -0.001861, -0.0183702,
 -0.1344516, -0.0805426, 0.071469, 0.09303741, 0.152423, 0.18703242,
 0.0747038, -0.0508168, 0.1444946, -0.0035698, -0.08759481, -0.076339,
 0.12213258, 0.0536314, 0.0848024, -0.0126872, 0.0621534, -0.0136734,
 0.0460204, 0.0165238, -0.0114024, 0.0789194, -0.12512961, -0.0599654,
 0.0271274, 0.135197, 0.1203802, 0.012631, -0.10814039, -0.001666,
 0.102235, 0.1412142, -0.1432326, 0.0364698, -0.0405188, -0.0619444,
 -0.0492014, -0.0717724, -0.1767314, 0.0941186, -0.0390486, 0.00346881,
 -0.0504774, -0.1277366, -0.1068394, -0.016379, 0.08483, 0.0436618,
 -0.0290356, -0.1612932, -0.1624266, 0.0509808, -0.0009336, -0.06578721,
 0.064263, 0.0331882, -0.0223034, 0.15584101, 0.0763886, 0.167244,
 0.0055764, -0.0992348, 0.0563692, 0.0334334, 0.0829898, -0.0114544,
 0.0827342, 0.003771, 0.11818061, -0.1387858, -0.16577761, -0.0435748,
 -0.0652956, 0.1459164, -0.092326, 0.0457522, 0.1035466, 0.11414399,
 0.0691928, -0.07784601, 0.0418476, 0.0170276, 0.0658464, -0.0077656,
 0.0068582, 0.0546224, 0.09042339, -0.0248642, 0.12831381, -0.07216899,
 -0.0140408, 0.0875768, -0.1052142, -0.0194332, -0.010973, -0.0451834,
 -0.0675572, 0.0428458, -0.0641458, 0.12179039, 0.0664384, -0.107638,
 0.17734359, -0.053135, 0.0106202, -0.0647184, -0.053587, 0.0073988,
 0.1080248, -0.0046844, -0.0360702, -0.0523194, -0.0375274, -0.0285122,
 0.0196288, 0.041194, 0.1889396, -0.096296, 0.0562956, 0.02518399,
 -0.0009708, -0.0854838, 0.1022352, 0.062502, -0.0499102, 0.1525736,
 0.1061274, 0.0242386, 0.0211244, -0.0302766, 0.18772559, -0.0856622,
 0.207165, 0.0609228, -0.0187748, -0.0456716, 0.12414519, 0.109781,
 0.0735934, 0.16670501, -0.0586932, -0.2012252, 0.06886339, 0.0472362,
 -0.24177119, 0.1424724, -0.0371788, 0.0703512, -0.0885094, -0.0168308,
 0.0459068, 0.1166916, -0.01727881, 0.0551434, 0.0373472, -0.0564362,
 0.1323012, 0.0090224, 0.1767146, 0.19905919, -0.0011484, 0.09186,
 -0.0228936, 0.0593226, 0.1362142, -0.0113414, -0.0417284, 0.08379,
 -0.0330084, -0.03994419, -0.0810144, 0.106915, 0.18364678, -0.0166938,
 -0.13166079, -0.0920856, 0.059012, 0.1711318, -0.08764941, -0.0141916,
 -0.11621161, -0.006381, 0.06006581, -0.1001544, -0.05137841, 0.1269888,
 0.052793, 0.23856762, 0.073796, 0.0570218, -0.0531292, -0.16851,
 0.0124098, -0.0032158, 0.053224, -0.0349538, 0.0697724, -0.2481112,
 -0.12974378, 0.0010186, -0.11761081, 0.13135299, -0.0193144, -0.0770366,
 0.070598, -0.0012104, -0.1015162, 0.05923041, 0.11690021, -0.13229279,
 0.0367152, -0.11344141, 0.1702454, 0.0231894, 0.0874332, 0.04045121,
 -0.0516392, -0.0322326, 0.0095716, -0.0441072, 0.0474756, -0.0242582,
 0.0439002, 0.0247672, 0.0913422, 0.01262119, 0.1575518, -0.039768  ])

if np.allclose(teste_vetorizar, vetor_esperado):
    print("Teste passou!")
else:
    print("Teste falhou.")

Teste passou!


### 3.2.8 Label Encoder
Etapa para realizar a conversão dos valores categóricos das em numéricos.

In [67]:
def label_encoding(df, coluna):
    le = LabelEncoder()
    df[coluna] = le.fit_transform(df[coluna])
    return df, le

####Teste da etapa

In [68]:
dados_teste = {'classificacao': ['positivo', 'negativo', 'positivo', 'negativo', 'positivo']}
df_teste = pd.DataFrame(dados_teste)

teste_label_encoder = label_encoding(df_teste, 'classificacao')
print(teste_label_encoder)

classes_esperadas = np.array([1, 0, 1, 0, 1])

if np.array_equal(teste_label_encoder, classes_esperadas):
    print("Teste passou!")
else:
    print("Teste falhou.")


(   classificacao
0              1
1              0
2              1
3              0
4              1, LabelEncoder())
Teste falhou.


### 3.2.9 Pipeline de pré-processamento
Apresenta as etapas em sequência, automatizando o fluxo de pré-processamento dos inputs e dataset.

Isso é realizado combinando as funções definidas anteriormente: remover pontuação e datas, converter o texto para minúsculas, tokenizar, remover stopwords, e vetorizar os tokens. Para o dataset, é também aplicado a codificação das colunas categóricas.

In [69]:
def pipeline_frases(texto, model):
    somente_texto = remover_pontuacao_e_datas(texto)

    texto_minusculo = texto_para_minusculo(somente_texto)

    tokens = tokenizar_texto(texto_minusculo)

    texto_sem_stopwords = remover_stopwords(tokens)

    vetor = vetorizar(texto_sem_stopwords, model)

    return vetor

In [80]:
def pipeline_modelo(df, coluna_frase, coluna_classe, model):
    df[coluna_frase] = df[coluna_frase].apply(lambda frase: pipeline_frases(frase, model))

    df, le = label_encoding(df, coluna_classe)

    return df, le


####Teste da etapa

In [81]:
pipeline_frase_teste = pipeline_frases('O rato roeu a roupa do rei de Roma', model_w2v)

vetor_esperado = np.array([0.011315, -0.0205364, -0.17001821, 0.0281516, -0.0009176, 0.0048138,
 0.18003121, -0.080958, 0.10080741, 0.0004238, -0.1151844, -0.19962598,
 0.0086752, 0.1176168, -0.1799024, -0.0301898, -0.0458002, 0.13258919,
 -0.2186378, -0.0577094, -0.0155758, -0.1074546, 0.0311892, 0.0110948,
 -0.0331072, -0.0066584, -0.0006306, 0.0363484, -0.03908, -0.0272782,
 0.0033082, 0.12251041, -0.0291818, -0.0924454, 0.1087174, -0.0194612,
 -0.1306488, -0.0064442, -0.14920822, 0.0102678, 0.0451484, 0.12865861,
 0.0048968, 0.0209104, 0.1455828, 0.1445082, 0.0672162, -0.14650841,
 0.0122242, 0.0641984, -0.14387742, -0.12351298, -0.0743102, 0.010645,
 -0.0058324, 0.1283564, 0.17453358, -0.0113002, 0.07685401, -0.0293362,
 0.235621, 0.1729084, -0.0046264, 0.2931738, 0.0711944, 0.1012754,
 0.0191284, -0.0504944, -0.056211, 0.11169519, -0.001861, -0.0183702,
 -0.1344516, -0.0805426, 0.071469, 0.09303741, 0.152423, 0.18703242,
 0.0747038, -0.0508168, 0.1444946, -0.0035698, -0.08759481, -0.076339,
 0.12213258, 0.0536314, 0.0848024, -0.0126872, 0.0621534, -0.0136734,
 0.0460204, 0.0165238, -0.0114024, 0.0789194, -0.12512961, -0.0599654,
 0.0271274, 0.135197, 0.1203802, 0.012631, -0.10814039, -0.001666,
 0.102235, 0.1412142, -0.1432326, 0.0364698, -0.0405188, -0.0619444,
 -0.0492014, -0.0717724, -0.1767314, 0.0941186, -0.0390486, 0.00346881,
 -0.0504774, -0.1277366, -0.1068394, -0.016379, 0.08483, 0.0436618,
 -0.0290356, -0.1612932, -0.1624266, 0.0509808, -0.0009336, -0.06578721,
 0.064263, 0.0331882, -0.0223034, 0.15584101, 0.0763886, 0.167244,
 0.0055764, -0.0992348, 0.0563692, 0.0334334, 0.0829898, -0.0114544,
 0.0827342, 0.003771, 0.11818061, -0.1387858, -0.16577761, -0.0435748,
 -0.0652956, 0.1459164, -0.092326, 0.0457522, 0.1035466, 0.11414399,
 0.0691928, -0.07784601, 0.0418476, 0.0170276, 0.0658464, -0.0077656,
 0.0068582, 0.0546224, 0.09042339, -0.0248642, 0.12831381, -0.07216899,
 -0.0140408, 0.0875768, -0.1052142, -0.0194332, -0.010973, -0.0451834,
 -0.0675572, 0.0428458, -0.0641458, 0.12179039, 0.0664384, -0.107638,
 0.17734359, -0.053135, 0.0106202, -0.0647184, -0.053587, 0.0073988,
 0.1080248, -0.0046844, -0.0360702, -0.0523194, -0.0375274, -0.0285122,
 0.0196288, 0.041194, 0.1889396, -0.096296, 0.0562956, 0.02518399,
 -0.0009708, -0.0854838, 0.1022352, 0.062502, -0.0499102, 0.1525736,
 0.1061274, 0.0242386, 0.0211244, -0.0302766, 0.18772559, -0.0856622,
 0.207165, 0.0609228, -0.0187748, -0.0456716, 0.12414519, 0.109781,
 0.0735934, 0.16670501, -0.0586932, -0.2012252, 0.06886339, 0.0472362,
 -0.24177119, 0.1424724, -0.0371788, 0.0703512, -0.0885094, -0.0168308,
 0.0459068, 0.1166916, -0.01727881, 0.0551434, 0.0373472, -0.0564362,
 0.1323012, 0.0090224, 0.1767146, 0.19905919, -0.0011484, 0.09186,
 -0.0228936, 0.0593226, 0.1362142, -0.0113414, -0.0417284, 0.08379,
 -0.0330084, -0.03994419, -0.0810144, 0.106915, 0.18364678, -0.0166938,
 -0.13166079, -0.0920856, 0.059012, 0.1711318, -0.08764941, -0.0141916,
 -0.11621161, -0.006381, 0.06006581, -0.1001544, -0.05137841, 0.1269888,
 0.052793, 0.23856762, 0.073796, 0.0570218, -0.0531292, -0.16851,
 0.0124098, -0.0032158, 0.053224, -0.0349538, 0.0697724, -0.2481112,
 -0.12974378, 0.0010186, -0.11761081, 0.13135299, -0.0193144, -0.0770366,
 0.070598, -0.0012104, -0.1015162, 0.05923041, 0.11690021, -0.13229279,
 0.0367152, -0.11344141, 0.1702454, 0.0231894, 0.0874332, 0.04045121,
 -0.0516392, -0.0322326, 0.0095716, -0.0441072, 0.0474756, -0.0242582,
 0.0439002, 0.0247672, 0.0913422, 0.01262119, 0.1575518, -0.039768  ])
if np.allclose(pipeline_frase_teste, vetor_esperado):
    print("Teste passou!")
else:
    print("Teste falhou.")

Teste passou!


### 3.2.10 Processamento da base de dados

Essa etapa aplica o pré-processamento da base de dados

In [84]:
df_processado, le = pipeline_modelo(df_modelo1, 'frase', 'classificacao', model_w2v)

In [86]:
df_processado.head()

,frase,classificacao
0,"[0.041368168, -0.09500485, -0.0861235, -0.0280...",2
1,"[0.019412603, 0.050926, 0.012795602, 0.0225042...",2
2,"[0.16175881, -0.1330094, -0.0760382, 0.0141349...",2
3,"[0.02117625, -0.18160576, -0.041077998, 0.0855...",2
4,"[0.0429665, 0.020914251, -0.13752425, 0.089154...",2


## 3.3 Treinamento do modelo

Etapa em que o modelo identifica padrões para aprender a receber entradas e mapear as saídas corretas.

O código separa as entradas e saídas esperadas, e em seguida divide o conjunto em dados de treino e teste.

In [87]:
X = np.vstack(df_processado['frase'])
y = df_processado['classificacao']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = RandomForestClassifier()
clf.fit(X_train, y_train)


RandomForestClassifier()

O modelo prevê as classes para os dados de teste.

As métricas do modelo revelam um overfitting, situação esperada devido ao dataset com dados artificiais, demonstrando pouca variabilidade e consequentemente aumento da previsibilidade.

Um relatório completo das métricas pode ser encontrado na documentação do projeto (index.md).

In [88]:
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.95      0.97        19
           1       0.94      1.00      0.97        17
           2       1.00      1.00      1.00        19

    accuracy                           0.98        55
   macro avg       0.98      0.98      0.98        55
weighted avg       0.98      0.98      0.98        55



## 3.4 Previsão
Etapa responsável por inferenciar uma classe (rótulo) em novos dados.

In [89]:
def prever(input):
    classe_prevista = clf.predict(input.reshape(1, -1))
    classe_original = le.inverse_transform(classe_prevista)

    return classe_original.tolist() if isinstance(classe_original, np.ndarray) else classe_original

A função `pesquisa` atua como pipeline, reunindo o pipeline de pré-processamento, extração de datas, reguladores e função de previsão.

É a sequência de todas as etapas que o input passa até ocorrer a previsão.

In [90]:
def pesquisa(input):
  input_processado = pipeline_frases(input, model_w2v)
  datas = encontrar_datas(input)
  texto_minusculo = texto_para_minusculo(input)
  reguladores = extrair_reguladores(texto_minusculo)
  previsao = prever(input_processado)

  return previsao, datas, reguladores

###Teste da etapa

In [91]:
teste_pesquisa = pesquisa('Leis sobre comunhão de recursos da anbima de 20/08/2003')
print(teste_pesquisa)

if teste_pesquisa == (['Finanças'], ['20/08/2003'], ['anbima']):
    print("Teste passou!")
else:
    print("Teste falhou.")

(['Fundos de investimento'], ['20/08/2003'], ['anbima'])
Teste falhou.


# 4. Modelo BERT



## 4.1 Dataset

In [92]:
df_modelo2 = df_dados.copy()
df_modelo2.head()

,frase,classificacao
0,Quero todas as regulamentações sobre fundos de...,Fundos de investimento
1,Quais são as normas para criação de fundos de ...,Fundos de investimento
2,Preciso das regulamentações sobre fundos imobi...,Fundos de investimento
3,Informações sobre fundos de índice.,Fundos de investimento
4,Quero as regras para fundos de ações.,Fundos de investimento



## 4.2 Pré-processamento do modelo BERT

Com a base de dados pronta, agora é necessário carregar o tokenizador BERT pré-treinado. O modelo utilizado nessa caso é o `bert-base-portuguese-cased`, que é específico para o português.

O tokenizador é responsável por converter as frases em tokens, que vão ser utilizados pelo modelo.

In [93]:
def get_tokenizer(model_name='neuralmind/bert-base-portuguese-cased'):
    return BertTokenizer.from_pretrained(model_name)

É necessário criar uma classe personalizada para criar um dataset compatível com o PyTorch, criado a partir das codificações e labels, e retornando os itens em formato de tensor. Para isso, utilizamos a classe `Dataset` do PyTorch, que garantirá que os dados estejam na maneira esperada para iteração no treinamento.

In [94]:
class CustomTextDataset(Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

A função `tokenize_function` serve para aplicar a tokenização nos textos. O uso de truncamento e padding é feito para garantir que todas as sequências de tokens tenham o mesmo comprimento (128).

In [95]:
def tokenize_texts(tokenizer, texts, max_length=128):
    return tokenizer(texts, padding='max_length', truncation=True, max_length=max_length)

Já é importante ter um meio para transformar os dados categóricos em numéricos, que será utilizado nos próximos passos.

In [96]:
def map_labels(labels):
    label_to_idx = {label: idx for idx, label in enumerate(set(labels))}
    labels_idx = [label_to_idx[label] for label in labels]
    return labels_idx, label_to_idx

O próximo passo é dividir o dataset em conjunto de treino e teste, tomando o cuidado para manter a proporção das classes. Pelo tamanho do dataset, apenas 10% dos dados serão usados para validação. Dessa forma, é possível tokenizar as frases dos conjuntos que foram divididos.

Agora que os dados foram preparados, podemos criar o dataset utilizado para o treinamento. Para isso, os conjuntos de treino e teste serão transformados em instâncias da classe `CustomTextDataset`.

In [97]:
def prepare_dataset(df, tokenizer, test_size=0.1, max_length=128):
    train_texts, val_texts, train_labels, val_labels = train_test_split(
        df['frase'].tolist(), df['classificacao'].tolist(),
        test_size=test_size, stratify=df['classificacao']
    )

    train_encodings = tokenize_texts(tokenizer, train_texts, max_length)
    val_encodings = tokenize_texts(tokenizer, val_texts, max_length)

    train_labels, label_to_idx = map_labels(train_labels)
    val_labels = [label_to_idx[label] for label in val_labels]

    train_dataset = CustomTextDataset(train_encodings, train_labels)
    val_dataset = CustomTextDataset(val_encodings, val_labels)

    return train_dataset, val_dataset, label_to_idx


## 4.3 Treinamento do modelo BERT

É preciso carregar o modelo BERT pré-treinado para classificação de sequências, em que é indicado o número de classes no dataset.

Durante o treinamento, o modelo consumirá o dataset dividido em vários batches menores. Um **batch** é um grupo de amostras que é processado de uma vez.

Uma **época** é um ciclo completo pelo dataset, ou seja, quando todas as amostras forem processadas.

Nesta célula, são definidos os argumentos de treinamento, como o tamanho do batch, o número de épocas, e os diretórios para salvar os resultados e logs.




É criado o objeto `Trainer`, que será responsável por dirigir o treinamento do modelos, usando os datasets de treino e testes.

Por fim, o modelo é treinado com os dados de treino, utilizando os parâmentros e datasets definidos, ajustando o BERT para a tarefa específica.

In [98]:
def train_model(train_dataset, val_dataset, num_labels, model_name='neuralmind/bert-base-portuguese-cased', epochs=2):
    model = BertForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

    training_args = TrainingArguments(
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        output_dir='./results',
        num_train_epochs=epochs,
        logging_dir='./logs',
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
    )

    trainer.train()

    return model, trainer

O modelo treinado é avaliado utilizando o conjunto de teste e retornando as métricas de desempenho.

O `eval_loss` indica a perda no conjunto de avaliação. Quanto menor a perda, melhor o desempenho do modelo.

Como o dataset foi construído com dados artificiais, o indicador baixo do eval_loss pode indicar que o conjunto de validação é muito fácil.

O restante das métricas `eval_runtime`, `eval_samples_per_second` e  `eval_steps_per_second` informam o número de amostras e épocas procesadas por segundo e o tempo total para avaliação do modelo. Devido ao tamanho do dataset, esses números foram relativamente baixos, informando que o modelo está avaliando os dados de forma rápida.

In [99]:
def evaluate_model(trainer, val_dataset):
    eval_results = trainer.evaluate(eval_dataset=val_dataset)
    return eval_results


### 4.3.1 Pipeline de treinamento

Sequencia as etapas de treinamento do modelo, apresenta as métricas do modelo e configura o salvamento.

In [100]:
def pipeline_treino(df):
  tokenizer = get_tokenizer()
  train_dataset, val_dataset, label_to_idx = prepare_dataset(df, tokenizer)
  model, trainer = train_model(train_dataset, val_dataset, num_labels=len(label_to_idx))

  model.save_pretrained('./saved_model')
  tokenizer.save_pretrained('./saved_model')

  with open('label_to_idx.pkl', 'wb') as f:
        pickle.dump(label_to_idx, f)

  return model, trainer, label_to_idx, val_dataset

### 4.3.2 Treinando, salvando e apresentando métricas do modelo

In [101]:
model, trainer, label_to_idx, val_dataset = pipeline_treino(df_modelo2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss


In [102]:
eval_results = evaluate_model(trainer, val_dataset)
print(eval_results)

{'eval_loss': 0.0406285859644413, 'eval_runtime': 15.4962, 'eval_samples_per_second': 1.807, 'eval_steps_per_second': 0.258, 'epoch': 2.0}


## 4.5 Previsão
Etapa responsável por inferenciar uma classe (rótulo) em novos dados.

Para preparar os novos dados de entrada, os que não foram vistos durante o treinamento, é necessário definir uma nova classe para organizar os dados para serem processados. Utilizamos o tokenizador no input (inserido na seção 'input' desse notebook) para novamente preparar os dados para passar no modelo.

O input tokenizado agora é encapsulado em uma instância da classe `RealDataDataset`, semelhante ao passo realizado com os dados de treinamento.

O `model.eval()` é utilizado para colocar o modelo em modo de avaliação, desativando algumas operações que são utilizadas no treinamento.

Em seguida, o modelo faz as previsões sobre o input e a classe com maior probabilidade é selecionada.

Por fim, as previsões numéricas são convertidas de volta para texto utilizando o mapeamento que foi criado anteriormente.

In [103]:
def bert_prever(model, tokenizer, texto, label_to_idx, max_length=128):
    class RealDataDataset(Dataset):
        def __init__(self, encodings):
            self.encodings = encodings

        def __getitem__(self, idx):
            item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
            return item

        def __len__(self):
            return len(self.encodings['input_ids'])

    real_encodings = tokenizer(texto, truncation=True, padding=True, max_length=max_length, return_tensors='pt')
    real_dataset = RealDataDataset(real_encodings)

    model.eval()
    with torch.no_grad():
        predictions = model(**real_encodings).logits
        predicted_labels = torch.argmax(predictions, dim=1)

    idx_to_label = {idx: label for label, idx in label_to_idx.items()}
    decoded_predictions = [idx_to_label[pred.item()] for pred in predicted_labels]

    return decoded_predictions

## 4.6 Extração de datas e entidades

Etapa responsável por extrair as datas e reguladores do input.

### 4.6.1 Minúsculo

In [104]:
def texto_para_minusculo(texto):
    return texto.lower()

####Teste da etapa

In [105]:
resultado_minusculo = texto_para_minusculo('O rato roeu a roupa do rei de Roma')

if resultado_minusculo == "o rato roeu a roupa do rei de roma":
    print("Teste passou!")
else:
    print("Teste falhou.")

Teste passou!


### 4.6.2 Extrair Entidades Regulamentadoras

In [106]:
def extrair_reguladores(tokens):
    dicionario = {"anbima", "nuclea", "bacen", "cvm", "b3", "selic", "banco do brasil", "anpd", "bsm", "febraban", "apimec", "abbc", "coaf"}
    padrao = r'\b(?:' + '|'.join(re.escape(palavra) for palavra in dicionario) + r')\b'

    if isinstance(tokens, str):
        doc = nlp(tokens)
        tokens = [token.text for token in doc]

    reguladores_encontrados = [token for token in tokens if re.search(padrao, token.lower())]

    return reguladores_encontrados

####Testes da etapa

Teste com tokens

In [107]:
teste_reguladores_tokens = extrair_reguladores(['todas', 'as', 'regulamentações', 'do', 'bacen'])
print('Reguladores:', teste_reguladores_tokens)

if teste_reguladores_tokens == ['bacen']:
    print("Teste passou!")
else:
    print("Teste falhou.")

Reguladores: ['bacen']
Teste passou!


Teste com frase

In [108]:
teste_reguladores_string = extrair_reguladores('todas as regulamentações do bacen')
print('Reguladores:', teste_reguladores_string)

if teste_reguladores_string == ['bacen']:
    print("Teste passou!")
else:
    print("Teste falhou.")

Reguladores: ['bacen']
Teste passou!


### 4.6.3 Extrair Datas

In [109]:
def encontrar_datas(texto):
    padrao_data = r'\b\d{1,2}/\d{1,2}(?:/\d{2,4})?\b'
    datas = re.findall(padrao_data, texto)
    return datas

####Testes da etapa

In [110]:
teste_datas = encontrar_datas('O rato roeu a roupa do rei de Roma em 14/04/2004 e depois em 12/08 do mesmo ano.')
print('Datas:', teste_datas)

if teste_datas == ['14/04/2004', '12/08']:
    print("Teste passou!")
else:
    print("Teste falhou.")

Datas: ['14/04/2004', '12/08']
Teste passou!


## 4.7 Encontrar tags similares

Encontra as tags mais similares à classe prevista

Define as tags que existem

In [111]:
tags = ['crédito', 'custódia', 'Resolução CVM 175', 'investidores profissionais', 'cedente', 'Banking as a Service', 'CCBs', 'gestão de fundos', 'supervisão de securitização']

Função para configurar a vetorização. Utiliza o modelo Word2Vec carregado anteriormente nesse notebook (seção 3.2.7)

In [112]:
def vetorizar(tokens, modelo):
    vetores = [modelo[token] for token in tokens if token in modelo]

    if len(vetores) > 0:
        return np.mean(vetores, axis=0)
    else:
        return np.zeros(modelo.vector_size)

Essa função aplica a vetorização na classe prevista e nas tags.

In [113]:
def check_vectors(modelo, prediction, tags):
  class_vetor = vetorizar(prediction[0], modelo)

  tags_vetors = np.array([vetorizar(tag, modelo) for tag in tags])

  if np.all(class_vetor == 0):
      print("Erro: o vetor da classe é um vetor de zeros. A palavra pode não estar no vocabulário.")

  for tag, vetor in zip(tags, tags_vetors):
      if np.all(vetor == 0):
          print(f"Erro: o vetor da tag '{tag}' é um vetor de zeros. A palavra pode não estar no vocabulário.")
  return class_vetor, tags_vetors

Com as tags e a previsao vetorizadas, é possível calcular a similaridade do cosseno para encontrar as n tags mais similar à classe prevista. Isso auxiliará em uma resposta de regulamentações mais abrangente para as pesquisas realizadas.

In [114]:
def pegar_similaridades(class_vetor, tags_vetors, tags, n=5):
    similaridades = cosine_similarity([class_vetor], tags_vetors)[0]

    top_indices = np.argsort(similaridades)[-n:][::-1]

    top_tags = [tags[i] for i in top_indices]
    top_similaridades = [similaridades[i] for i in top_indices]

    return list(zip(top_tags, top_similaridades))

## 4.8 Pipeline pesquisa

Sequencia as etapas para receber um input e obter a previsão, datas, reguladores e tags similares com o modelo BERT.

A função `resultado` atua como pipeline, novamente condensando as funções apresentadas anteriormente, com o objetivo de automatizar o processo.

In [115]:
def resultado(texto, tags):

    tokenizer = get_tokenizer()
    previsao = bert_prever(model, tokenizer, texto, label_to_idx)

    class_vetor, tags_vetors = check_vectors(model_w2v, previsao, tags)
    top_similaridades = pegar_similaridades(class_vetor, tags_vetors, tags)

    datas = encontrar_datas(texto)
    texto_minusculo = texto_para_minusculo(texto)
    reguladores = extrair_reguladores(texto_minusculo)

    return previsao, datas, reguladores, top_similaridades

### Teste da etapa

In [116]:
resultado_teste = resultado('textos de dinheiro do bacen de 20/09/2003', tags)
print(resultado_teste)

if resultado_teste == (['Finanças'], ['20/09/2003'], ['bacen'], [('Banking as a Service', 0.94539165), ('supervisão de securitização', 0.8893843), ('custódia', 0.8556227), ('investidores profissionais', 0.85392517), ('gestão de fundos', 0.84931135)]):
    print("Teste passou!")
else:
    print("Teste falhou.")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


(['Finanças'], ['20/09/2003'], ['bacen'], [('Banking as a Service', 0.89681077), ('investidores profissionais', 0.8259202), ('gestão de fundos', 0.80301666), ('cedente', 0.7928282), ('supervisão de securitização', 0.7379688)])
Teste falhou.
